In [1]:
import os
print(os.getcwd())


j:\Chapter\IEEE-CS\ml_domain_transfer\archive (1)


In [4]:
import os
import cv2
import numpy as np

# ================= CONFIG =================
INPUT_ROOT = r"J:\Chapter\IEEE-CS\ml_domain_transfer\archive (1)\competition"
OUTPUT_ROOT = r"J:\Chapter\IEEE-CS\frames_new"
NUM_FRAMES = 16
IMG_SIZE = 224
VIDEO_EXT = ".mp4"
# =========================================


def extract_frames(video_path, out_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[ERROR] Cannot open {video_path}")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames <= 0:
        print(f"[SKIP] No frames in {video_path}")
        cap.release()
        return

    indices = np.linspace(0, total_frames - 1, NUM_FRAMES).astype(int)

    os.makedirs(out_dir, exist_ok=True)

    saved = 0
    last_frame = None

    frame_id = 0
    idx_ptr = 0

    while cap.isOpened() and idx_ptr < NUM_FRAMES:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id == indices[idx_ptr]:
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            out_path = os.path.join(out_dir, f"{idx_ptr}.jpg")
            cv2.imwrite(out_path, frame)
            last_frame = frame
            saved += 1
            idx_ptr += 1

        frame_id += 1

    cap.release()

    # Pad with last frame if video was too short
    while saved < NUM_FRAMES and last_frame is not None:
        out_path = os.path.join(out_dir, f"{saved}.jpg")
        cv2.imwrite(out_path, last_frame)
        saved += 1



def process_split(split):
    print(f"\nProcessing {split} set")

    in_base = os.path.join(INPUT_ROOT, split)
    out_base = os.path.join(OUTPUT_ROOT, split)

    if split == "test":
        videos = os.listdir(in_base)
        for v in videos:
            if not v.endswith(VIDEO_EXT):
                continue
            vid_name = os.path.splitext(v)[0]
            extract_frames(
                os.path.join(in_base, v),
                os.path.join(out_base, vid_name)
            )
    else:
        for cls in ["fake", "real"]:
            in_cls = os.path.join(in_base, cls)
            out_cls = os.path.join(out_base, cls)

            for v in os.listdir(in_cls):
                if not v.endswith(VIDEO_EXT):
                    continue
                vid_name = os.path.splitext(v)[0]
                extract_frames(
                    os.path.join(in_cls, v),
                    os.path.join(out_cls, vid_name)
                )


def main():
    process_split("train")
    process_split("test")
    print("\n✅ Frame extraction complete. Every video has EXACTLY 16 frames.")


if __name__ == "__main__":
    main()



Processing train set

Processing test set

✅ Frame extraction complete. Every video has EXACTLY 16 frames.


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import pandas as pd
import os

# ================= CONFIG =================
TEST_FRAMES_ROOT = r"J:\Chapter\IEEE-CS\frames_new\test"
MODEL_PATH = 'deepfake_model.pth'
CSV_OUTPUT_PATH = 'submission_test_predictions.csv'
FRAMES = 16
BATCH_SIZE = 4
IMG_SIZE = 224
# =========================================

# Dataset returning (video_name, frames_tensor)
class TestVideoFrameDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for video_dir in sorted(os.listdir(root_dir)):
            video_path = os.path.join(root_dir, video_dir)
            if os.path.isdir(video_path):
                frame_files = os.listdir(video_path)
                if len(frame_files) != FRAMES:
                    print(f"[WARN] {video_path} has {len(frame_files)} frames, skipping")
                    continue
                self.samples.append(video_dir)  # store only name

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_name = self.samples[idx]
        video_path = os.path.join(self.root_dir, video_name)
        frame_files = sorted(os.listdir(video_path))

        frames = []
        for f in frame_files:
            img = Image.open(os.path.join(video_path, f)).convert("RGB")
            if self.transform:
                img = self.transform(img)
            frames.append(img)

        x = torch.stack(frames)  # [16, 3, 224, 224]
        return video_name, x

# Transforms (same as training)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_dataset = TestVideoFrameDataset(TEST_FRAMES_ROOT, transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=0)

print(f"Test videos found: {len(test_dataset)}")

# Model (same as training)
class VideoMaxPoolModel(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        resnet = torch.hub.load('pytorch/vision:v0.10.0',
                                'resnet18', pretrained=pretrained)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])
        self.feature_dim = resnet.fc.in_features
        self.fc = nn.Linear(self.feature_dim, 1)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        feats = self.cnn(x)
        feats = feats.view(B, T, -1)
        feats, _ = torch.max(feats, dim=1)
        out = self.fc(feats)
        return torch.sigmoid(out).squeeze(1)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = VideoMaxPoolModel(pretrained=False).to(device)
checkpoint = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Inference and collection
video_names_all = []
preds_all = []
probs_all = []

with torch.no_grad():
    for video_names, x in test_loader:
        x = x.to(device)  # [B, 16, 3, 224, 224]
        probs = model(x)  # [B], in [0,1]

        preds = (probs > 0.5).long()  # 0 or 1

        video_names_all.extend(video_names)
        preds_all.extend(preds.cpu().tolist())
        probs_all.extend(probs.cpu().tolist())

# Build DataFrame in required format
df_sub = pd.DataFrame({
    "Video_Name": [v + ".mp4" for v in video_names_all],  # if they expect .mp4
    "Prediction": preds_all,
    "Probability": probs_all
})

df_sub.to_csv(CSV_OUTPUT_PATH, index=False)
print(f"\n✅ Saved predictions to: {CSV_OUTPUT_PATH}")
print(df_sub.head())


Test videos found: 100


Using cache found in C:\Users\johnp/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\johnp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\johnp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\johnp\AppData\Local\Temp\ipykernel_20688\3858377636.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/p


✅ Saved predictions to: submission_test_predictions.csv
                             Video_Name  Prediction  Probability
0  000e80813b5846bd9973b260934ae570.mp4           0     0.202184
1  005fca2f60a544b5b6a7af9d69e3ebcd.mp4           1     0.686167
2  00935c8593694510a2e6da69ad1ac70a.mp4           0     0.450471
3  022e20a3e98640e2a1399df87e375ada.mp4           0     0.384968
4  078c7f87286648c791c957157dc734a1.mp4           0     0.000003
